# Akses ke google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# cek spec colab hardware

In [ ]:
# !df -h
# !cat /proc/cpuinfo
# !cat /proc/meminfo

### Perhatian Jika code dijalankan ulang, maka model yang dihasilkan agar berbeda, karena library scikit learn mengambil sampel data training dan testing secara acak untuk modeling

# Import Library yang diperlukan

In [ ]:
from glob import glob
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import numpy as np
import sys
import pandas as pd
import numpy as np
import re   
from nltk.tokenize import regexp_tokenize
pd.options.display.max_colwidth=30000

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Menggabungkan semua data yang diperoleh

In [ ]:
home_dir=""
pd.read_csv(home_dir+"/dataset science/sci_1_3.csv")
alldata=glob(home_dir+"/dataset science/sci*.csv") + \
        glob(home_dir+"/dataset business/bis*.csv") + \
        glob(home_dir+"/dataset tech/tech*.csv") + \
        glob(home_dir+"/dataset health/hlt*.csv") + \
        glob(home_dir+"/dataset entertainment/ent*.csv")


# alldata
with open(home_dir+"/allcategory.csv", "wb") as outfile:
    for f in alldata:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

# Membaca data dan melihat info data

In [ ]:
home_dir=dir
data=pd.read_csv(home_dir+"/allcategory.csv")
data.columns=["judul","isi","tanggal","kategori"]
berita=data.drop_duplicates()
kategori=berita["kategori"].replace({"entertaiment":"entertainment"})
kategori.value_counts()
berita["kategori"]=kategori
berita["kategori"].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


business         290
entertainment    180
technology       151
science          121
health            71
Name: kategori, dtype: int64

In [ ]:
berita.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 813 entries, 0 to 3654
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     813 non-null    object
 1   isi       813 non-null    object
 2   tanggal   813 non-null    object
 3   kategori  813 non-null    object
dtypes: object(4)
memory usage: 31.8+ KB


#Membagi data menjadi lebih seimbang (Undersampling)

In [ ]:
# Shuffle the Dataset.
shuffled_df = berita.sample(frac=1,random_state=1)

# Put all the fraud class in a separate dataset.
health = shuffled_df.loc[shuffled_df['kategori'] == 'health']

#Randomly select 100 observations from the non-health (majority class)
df1 = shuffled_df.loc[shuffled_df['kategori'] == 'science'].sample(n=100,random_state=1)
df2 = shuffled_df.loc[shuffled_df['kategori'] == 'technology'].sample(n=100)
df3 = shuffled_df.loc[shuffled_df['kategori'] == 'entertainment'].sample(n=100,random_state=1)
df4 = shuffled_df.loc[shuffled_df['kategori'] == 'business'].sample(n=100,random_state=1)

# Concatenate both dataframes again
berita = pd.concat([health, df1,df2,df3,df4])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
raw_news=berita[["isi","kategori"]]

In [ ]:
raw_news['kategori'].value_counts()

science          100
entertainment    100
technology       100
business         100
health            71
Name: kategori, dtype: int64

# cek kualitas data

In [ ]:
pd.options.display.max_colwidth=30000
raw_news.head()

isi kategori
493                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Scientists want people to send them their wildlife experiences under the coronavirus lockdown. They are keen to hear recordings of dawn choruses, animals in unusual places, and views of the night sky without pollution. The Earth Project is a global citizen science study co-ordinated by scientists across UK universities.It hopes to showcase how nature has capitalised on reduced human activity during the pandemic. The researchers want the public to help them capture a global representation of what we experienced on the ground during lockdown."We are hoping to create a useful shared library of baseline experiences for the public, reminding us in the future of what life and our relationship with nature could be when global public mobility and many pollution-generating activities are reduced," explained Phil Manning from the University of Manchester.The team acknowledges that "significant sacrifices" will have to be made by everyone in order to reduce our impact on the planet's climate system, and to move towards the sustainable use of the Earth's natural resources. It is hoped that the positive experiences many people had with nature during lockdown, and the recollection of those experiences, will help build the case for behaviour changes to help shift us towards sustainability.During the lockdown, wildlife reclaiming the streets of towns and cities during lockdown have made headlines around the world. For example, goats that normally keep themselves to themselves in the hills surrounding Llandudno, Wales, ventured into the town's deserted streets.People have also reported that they have been able to hear birdsong more clearly without the constant hum of traffic.The submissions will be collated by the Earth Project team, which consists of concerned citizens alongside scientists from multiple UK universities, including Bangor, Belfast, Durham, Manchester and Plymouth.Prof Manning told BBC News that the recordings of birdsong, night skies and nature calls will be made available online for everyone to enjoy and to relive the better moments of life under lockdown."The survey results will help provide an overview of people's observations and thoughts on the environment before, during and after lockdown," he observed."The responses will be analysed and summarised and then made available along with the database on our website during the late Summer and Autumn of 2020."He said that it was hoped that the find

In [ ]:
raw_news.tail()

isi  kategori
1111                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Foreign visitors to Spain will no longer have to undergo a two-week quarantine from 1 July, the government has announced.It said the measure had been finalised in a cabinet meeting on Monday.Foreign Minister Arancha Gonzalez Laya had previously said the requirement would be lifted in July, without giving a date.The news comes as the UK government prepares to bring in its own 14-day quarantine policy from 8 June.Travel firms and other industry bodies say the UK should relax the measure for visitors arriving from countries where people are at a lower risk of contracting the coronavirus.Spain normally attracts 80 million tourists a year, with the sector providing more than 12% of the country's GDP.Opening up the holiday market again before the summer season is over is seen as crucial to the Spanish economy. But under the UK's new policy, any tourists returning home after taking holidays in Spain and most other foreign destinations would have to spend two weeks in self-isolation.Several airlines including EasyJet, Jet2 and Ryanair have announced that they plan to resume flights and holidays soon.Easyjet will be resuming flights

In [ ]:
# raw_news[raw_news["kategori"]=="health"].head()

In [ ]:
# raw_news[raw_news["kategori"]=="entertainment"].head()

In [ ]:
# raw_news[raw_news["kategori"]=="technology"].head()

# Noise Removal
### setelah dilihat masih banyak kata-kata yang seharusnya tidak ikut dalam berita seperti follow* on twitter, *@bbc.co.uk, etc

langkah-langkah penyelesaiannya:
- masuk ke string
- pisahin dengan titik, buat list
- jika ada noise hapus
- gabungin lagi ke string

In [ ]:
stop=['Recent reviews by.*','.*Follow.*','.*follow.*','.*Follow.* on .+', '.*follow.* on .+','.*@bbc.*',
      '.*What questions do you have about.*', '.*displaying your name.*', '.*age and location as you provide it.*', 
      '.*unless you state otherwise.*','.*@BBC.*',
      '.*Please include your name.*','.*Your contact details.*','.*please subscribe.*',
      '.*Please ensure you have read our terms.*']
# raw_news['tokenized_dot'] = raw_news.apply(lambda row: sent_tokenize(row['isi']), axis=1)
raw_news['tokenized_dot'] = raw_news.apply(lambda row: row['isi'].split("."), axis=1)

kata=[]
for word in raw_news['tokenized_dot']:
  if re.search(r'|'.join(stop), str(word[-20:-1]) or str(word[-1]) ):
    for x in word:
      if re.search(r'|'.join(stop), x):
        kata.append(x)
raw_news['isi'] = raw_news['tokenized_dot'].apply(lambda x: ' '.join([word for word in x if word not in (kata)]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
raw_news.head()

isi  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
raw_news=raw_news[['isi','kategori']]

# Hapus sisa noise

In [ ]:
def remove_noise(sentence):
  sentence = sentence.replace("co uk","")
  sentence=re.sub('\w+ co','',sentence)
  return sentence

In [ ]:
raw_news['isi']=raw_news['isi'].apply(remove_noise)

In [ ]:
raw_news.head()

isi kategori
493                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Scientists want people to send them their wildlife experiences under ronavirus lockdown  They are keen to hear recordings of dawn choruses, animals in unusual places, and views of the night sky without pollution  The Earth Project is a global citizen science -ordinated by scientists across UK universities It hopes to showcase how nature has capitalised on reduced human activity during the pandemic  The researchers want the public to help them capture a global representation of what we experienced on the ground during lockdown "We are hoping to create a useful shared library of baseline experiences for the public, reminding us in the future of what life and our relationship with uld be when global public mobility and many pollution-generating activities are reduced," explained Phil Manning from the University of Manchester The team acknowledges that "significant sacrifices" will have to be made by everyone in order to reduce our impact on the planet's climate system, and to move towards the sustainable use of the Earth's natural resources  It is hoped that the positive experiences many people had with nature during lockdown, and the recollection of those experiences, will help build the case for behaviour changes to help shift us towards sustainability During the lockdown, wildlife reclaiming the streets of towns and cities during lockdown have made headlines around the world  For example, goats that normally keep themselves to themselves in the hills surrounding Llandudno, Wales, ventured into the town's deserted streets People have also reported that they have been able to hear birdsong more clearly without nstant hum of traffic The submissions will llated by the Earth Project team, nsists ncerned citizens alongside scientists from multiple UK universities, including Bangor, Belfast, Durham, Manchester and Plymouth Prof Manning told BBC News that the recordings of birdsong, night skies and nature calls will be made available online for everyone to enjoy and to relive the better moments of life under lockdown "The survey results will help provide an overview of people's observations and thoughts on the environment before, during and after lockdown," he observed "The responses will be analysed and summarised and then made available along with the database on our website during the late Summer and Autumn of 2020 "He said that it was hoped that the findings of the uld encourage behaviour change and help shape policies and strategies uld deliver greener and sustainable lifestyles "The Earth Project… has the potential to provide a snapshot of what happened to the environment and wildlife when human

# lowercasing

In [ ]:
#lowercasing
def remove_punc(sentence):
  sentence = sentence.lower()
  sentence = re.sub('[^a-z]+', ' ', sentence)
  return sentence

In [ ]:
raw_news['isi']=raw_news['isi'].apply(remove_punc)

In [ ]:
raw_news['kategori'].value_counts()

business         100
technology       100
entertainment    100
science          100
health            71
Name: kategori, dtype: int64

# Stopword removal

In [ ]:
sw = stopwords.words('english')
# sw.append('coronavirus')
def stop_word(sentence):
  new_sentence = []
  for word in sentence.split():
    if word not in sw:
      new_sentence.append(word)  
  return(' '.join(new_sentence))

raw_news['isi'] = raw_news['isi'].apply(stop_word)
raw_news.head(5)

,isi,kategori
493,scientists want people send wildlife experiences ronavirus lockdown keen hear recordings dawn choruses animals unusual places views night sky without pollution earth project global citizen science ordinated scientists across uk universities hopes showcase nature capitalised reduced human activity pandemic researchers want public help capture global representation experienced ground lockdown hoping create useful shared library baseline experiences public reminding us future life relationship uld global public mobility many pollution generating activities reduced explained phil manning university manchester team acknowledges significant sacrifices made everyone order reduce impact planet climate system move towards sustainable use earth natural resources hoped positive experiences many people nature lockdown recollection experiences help build case behaviour changes help shift us towards sustainability lockdown wildlife reclaiming streets towns cities lockdown made headlines around world example goats normally keep hills surrounding llandudno wales ventured town deserted streets people also reported able hear birdsong clearly without nstant hum traffic submissions llated earth project team nsists ncerned citizens alongside scientists multiple uk universities including bangor belfast durham manchester plymouth prof manning told bbc news recordings birdsong night skies nature calls made available online everyone enjoy relive better moments life lockdown survey results help provide overview people observations thoughts environment lockdown observed responses analysed summarised made available along database website late summer autumn said hoped findings uld encourage behaviour change help shape policies strategies uld deliver greener sustainable lifestyles earth project potential provide snapshot happened environment wildlife human activity drastically reduced simply managed sustainable way prof manning said part project survey ask people whether willing make changes behaviour help improve world around prof manning added information evaluated inclusive earth project think tank hopefully uld impact future behaviour guide make policy decisions earth project potential help catalyse humankind sustainable future access earth project live website,science
291,clear picture emerging asteroid struck earth million years ago catastrophic space object wiped species including dinosaurs hit worst possible place planet according new research lethal angle investigations crater site together mputer simulations suggest impactor dug crust inclination degrees exacerbated climatic fallout know target rocks gulf mexico contained huge volumes sulphur mineral gypsum material thrown high atmosphere mixed water vapour produced global winter angle attack ensured environmental crisis intense prolonged degrees impact efficient vaporising ejecting debris high altitude impact happens shallower much steeper angles amount material put atmosphere climate changing effects significantly less explained prof gareth collins imperial college london evident nature location event happened together impact angle made perfect storm told bbc news majority plant animal life earth succumbed nditions prof collins lleagues work published journal nature communications prof collins part international team studying anatomy crater associated calamitous asteroid strike today km wide structure positioned mexico yucatan peninsula best preserved central portions sitting offshore port chicxulub hard grasp scale forces produced impactor thought km diameter punched instantaneous hole crust probably km deep fluidised rocks base bowl rebounded created minutes mountain higher everest last however fell back leave prominent inner ring hills peaks interesting prof collins perspective asymmetry frozen chicxulub structure example look centres crater peak ring uplifted rock underlies crust earth mantle points map directly top actually aligned northeast southwest direction crater cen

# lemmatization

In [ ]:
#lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
  new_sentence = []
  for word in sentence.split():
    word = lemmatizer.lemmatize(word)
    new_sentence.append(word)
  return(' '.join(new_sentence))

raw_news['isi'] = raw_news['isi'].apply(lemmatize)
clean_news=raw_news
clean_news.head(5)

,isi,kategori
493,scientist want people send wildlife experience ronavirus lockdown keen hear recording dawn chorus animal unusual place view night sky without pollution earth project global citizen science ordinated scientist across uk university hope showcase nature capitalised reduced human activity pandemic researcher want public help capture global representation experienced ground lockdown hoping create useful shared library baseline experience public reminding u future life relationship uld global public mobility many pollution generating activity reduced explained phil manning university manchester team acknowledges significant sacrifice made everyone order reduce impact planet climate system move towards sustainable use earth natural resource hoped positive experience many people nature lockdown recollection experience help build case behaviour change help shift u towards sustainability lockdown wildlife reclaiming street town city lockdown made headline around world example goat normally keep hill surrounding llandudno wale ventured town deserted street people also reported able hear birdsong clearly without nstant hum traffic submission llated earth project team nsists ncerned citizen alongside scientist multiple uk university including bangor belfast durham manchester plymouth prof manning told bbc news recording birdsong night sky nature call made available online everyone enjoy relive better moment life lockdown survey result help provide overview people observation thought environment lockdown observed response analysed summarised made available along database website late summer autumn said hoped finding uld encourage behaviour change help shape policy strategy uld deliver greener sustainable lifestyle earth project potential provide snapshot happened environment wildlife human activity drastically reduced simply managed sustainable way prof manning said part project survey ask people whether willing make change behaviour help improve world around prof manning added information evaluated inclusive earth project think tank hopefully uld impact future behaviour guide make policy decision earth project potential help catalyse humankind sustainable future access earth project live website,science
291,clear picture emerging asteroid struck earth million year ago catastrophic space object wiped specie including dinosaur hit worst possible place planet according new research lethal angle investigation crater site together mputer simulation suggest impactor dug crust inclination degree exacerbated climatic fallout know target rock gulf mexico contained huge volume sulphur mineral gypsum material thrown high atmosphere mixed water vapour produced global winter angle attack ensured environmental crisis intense prolonged degree impact efficient vaporising ejecting debris high altitude impact happens shallower much steeper angle amount material put atmosphere climate changing effect significantly le explained prof gareth collins imperial college london evident nature location event happened together impact angle made perfect storm told bbc news majority plant animal life earth succumbed nditions prof collins lleagues work published journal nature communication prof collins part international team studying anatomy crater associated calamitous asteroid strike today km wide structure positioned mexico yucatan peninsula best preserved central portion sitting offshore port chicxulub hard grasp scale force produced impactor thought km diameter punched instantaneous hole crust probably km deep fluidised rock base bowl rebounded created minute mountain higher everest last however fell back leave prominent inner ring hill peak interesting prof collins perspective asymmetry frozen chicxulub structure example look centre crater peak ring uplifted rock underlies crust earth mantle point map directly top actually aligned northeast southwest direction crater centre centre mantle uplift peak ring formation vital clue determining directio

In [ ]:
# clean_news[clean_news['kategori']=='entertainment']

# Feature Extraction

In [ ]:
# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer
# Tfidf = TfidfVectorizer(ngram_range=(1, 2),max_df=0.5, min_df=0)
# Tfidf = TfidfVectorizer(ngram_range=(1, 2),max_df=0.30000000000000004, min_df=0)
Tfidf = TfidfVectorizer(ngram_range=(1, 2),max_df=0.4, min_df=0)

features = Tfidf.fit_transform(clean_news.isi).toarray() 

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5
Number = 1
for category in clean_news['kategori'].unique():
  features_chi2 = chi2(features, clean_news['kategori'] == category)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(Tfidf.get_feature_names())[indices]
  unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
  bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
  print('{}. {} :'.format(Number,category))
  print('\t# Unigrams :\n\t. {}'.format('\n\t. '.join(unigrams[-N:])))
  print('\t# Bigrams :\n\t. {}'.format('\n\t. '.join(bigrams[-N:])))
  Number += 1

1. science :
	# Unigrams :
	. hurley
	. spacex
	. astronaut
	. nasa
	. space
	# Bigrams :
	. climate change
	. space shuttle
	. crew dragon
	. hurley behnken
	. space station
2. technology :
	# Unigrams :
	. apps
	. facebook
	. game
	. app
	. apple
	# Bigrams :
	. app store
	. human right
	. death penalty
	. club penguin
	. apple google
3. health :
	# Unigrams :
	. vaccine
	. symptom
	. drug
	. trial
	. patient
	# Bigrams :
	. world health
	. immune system
	. clinical trial
	. covid patient
	. second wave
4. entertainment :
	# Unigrams :
	. art
	. theatre
	. music
	. rogan
	. song
	# Bigrams :
	. bbc three
	. night watch
	. joe rogan
	. end youtube
	. youtube post
5. business :
	# Unigrams :
	. trade
	. scheme
	. business
	. economy
	. airline
	# Bigrams :
	. royal mail
	. rare earth
	. furlough scheme
	. hong kong
	. self employed


# Modeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(clean_news['isi'], clean_news['kategori'])
#model=KNeighborsClassifier()
#dataset lengkap
# model=KNeighborsClassifier(algorithm='auto', leaf_size=30,
#                                       metric='euclidean', metric_params=None,
#                                       n_jobs=None, n_neighbors=29, p=2,
#                                       weights='distance')

#100 sample
model=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='euclidean', metric_params=None,
                                      n_jobs=None, n_neighbors=11, p=2,
                                      weights='distance')


In [ ]:
# create pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
                     ('vect',Tfidf),
                     ('clf',model)
])

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.4, max_features=None,
                                 min_df=0, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='euclidean', metric_params=None,


In [ ]:
#prediksi data test
# lembar jawaban kita
y_pred = pipeline.predict(X_test)

# Evaluation

In [ ]:
print(classification_report(y_pred,y_test)) # without hyperparameter tuning 100 sample

               precision    recall  f1-score   support

     business       0.83      0.89      0.86        28
entertainment       0.90      0.82      0.86        22
       health       0.94      0.89      0.92        19
      science       1.00      0.88      0.94        25
   technology       0.79      0.92      0.85        24

     accuracy                           0.88       118
    macro avg       0.89      0.88      0.88       118
 weighted avg       0.89      0.88      0.88       118



In [ ]:
print(classification_report(y_pred,y_test)) # without hyperparameter tuning

               precision    recall  f1-score   support

     business       0.92      0.86      0.89        78
entertainment       0.91      0.89      0.90        36
       health       0.80      0.94      0.86        17
      science       0.97      0.92      0.95        38
   technology       0.78      0.89      0.83        35

     accuracy                           0.89       204
    macro avg       0.88      0.90      0.89       204
 weighted avg       0.89      0.89      0.89       204



In [ ]:
print(classification_report(y_pred,y_test)) #with hyperparameter tuning

               precision    recall  f1-score   support

     business       0.89      0.90      0.90        72
entertainment       1.00      0.95      0.98        43
       health       1.00      0.91      0.95        22
      science       0.90      0.97      0.93        29
   technology       0.87      0.89      0.88        38

     accuracy                           0.92       204
    macro avg       0.93      0.93      0.93       204
 weighted avg       0.92      0.92      0.92       204



In [ ]:
print(classification_report(y_pred,y_test)) # with hyperparameter tuning 100 sample

               precision    recall  f1-score   support

     business       0.86      0.90      0.88        20
entertainment       1.00      0.96      0.98        25
       health       1.00      1.00      1.00        21
      science       1.00      0.94      0.97        33
   technology       0.86      0.95      0.90        19

     accuracy                           0.95       118
    macro avg       0.94      0.95      0.95       118
 weighted avg       0.95      0.95      0.95       118



In [ ]:
from sklearn.metrics import confusion_matrix #with tuning 100
import seaborn as sns
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[18,  1,  0,  1,  1],
       [ 0, 24,  0,  0,  0],
       [ 0,  0, 21,  0,  0],
       [ 0,  0,  0, 31,  0],
       [ 2,  0,  0,  1, 18]])

In [ ]:
from pycm import * #with tuning 100
cm = ConfusionMatrix(actual_vector=np.array(y_test), predict_vector=np.array(y_pred))
print(cm)

Predict             business            entertainment       health              science             technology          
Actual
business            18                  1                   0                   1                   1                   

entertainment       0                   24                  0                   0                   0                   

health              0                   0                   21                  0                   0                   

science             0                   0                   0                   31                  0                   

technology          2                   0                   0                   1                   18                  





Overall Statistics : 

95% CI                                                            (0.90951,0.98879)
ACC Macro                                                         0.97966
ARI                                                               0.88529
AUN

In [ ]:
from sklearn.metrics import confusion_matrix #without= tuning 100
import seaborn as sns
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


array([[25,  1,  0,  3,  1],
       [ 1, 18,  1,  0,  0],
       [ 0,  0, 17,  0,  1],
       [ 0,  0,  0, 22,  0],
       [ 2,  3,  1,  0, 22]])

In [ ]:
from sklearn.metrics import confusion_matrix #without tuning
import seaborn as sns
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


array([[67,  1,  0,  3,  2],
       [ 2, 32,  0,  0,  1],
       [ 3,  0, 16,  0,  1],
       [ 0,  1,  0, 35,  0],
       [ 6,  2,  1,  0, 31]])

In [ ]:
from sklearn.metrics import confusion_matrix #with tuning
import seaborn as sns
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[65,  2,  1,  1,  4],
       [ 0, 41,  0,  0,  0],
       [ 0,  0, 20,  0,  0],
       [ 3,  0,  0, 28,  0],
       [ 4,  0,  1,  0, 34]])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix # with tuning 100
multilabel_confusion_matrix(y_test, y_pred)

array([[[95,  2],
        [ 3, 18]],

       [[93,  1],
        [ 0, 24]],

       [[97,  0],
        [ 0, 21]],

       [[85,  2],
        [ 0, 31]],

       [[96,  1],
        [ 3, 18]]])

# Tuning Hyperparameter

In [ ]:
# grid search
from sklearn.model_selection import GridSearchCV
batas=[]
neigh=[]
for i in np.arange(0.1, 1.1, 0.1):
  batas.append(i)

for i in range(1,51):
  neigh.append(i)

print(batas,neigh)
params={'vect__max_df':batas,
        'clf__metric':['euclidean','manhattan'], 
        'clf__weights':['uniform','distance'],
        'clf__n_neighbors':neigh
        }

[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9, 1.0] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [ ]:
grid=GridSearchCV(pipeline,
                  params,
                  cv=5)

In [ ]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.30000000000000004,
                                                        max_features=None,
                                                        min_df=0,
                                                        ngram_range=(1, 2),
                                          

In [ ]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.4, max_features=None,
                                 min_df=0, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='euclidean', metric_params=None,


## simpan model

In [ ]:
# dump pipeline to pickle
import pickle
dir="/yourdir"
#pickle.dump(pipeline,open(dir+'final_class_sam.pkl', 'wb'))

# test model

In [ ]:
# load the model from disk
import pickle
#loaded_model = pickle.load(open(dir+'final_class_sam.pkl', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)
# loaded_model.predict([tes_bis, tes_tech, tes_sci, tes_ent, tes_hlt ])

0.9491525423728814


In [ ]:
tes_bis=readfile("bisnis.txt")
tes_tech=readfile("tech.txt")
tes_sci=readfile("science.txt")
tes_ent=readfile("entertainment.txt")
tes_hlt=readfile("health.txt")
loaded_model.predict([tes_bis, tes_tech, tes_sci, tes_ent, tes_hlt ])

array(['business', 'technology', 'science', 'entertainment', 'health'],
      dtype=object)

In [106]:
from collections import Counter
def readfile(name_file):
    with open(name_file) as f:
        file=f.read().splitlines()
        sentence=Counter(file)
        del(sentence[''])
        kalimat=""
        for x in sentence:
            kalimat+=x+" "
        return kalimat